In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2

In [2]:
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import matplotlib.pyplot as plt

In [7]:
annotation_train_path = 'data/cig_train_labels.csv'
image_path = 'data/cig/'
output_path =  'data/output/'
classes_of_interest = ['pick']

In [8]:
dtype_dict = {'filename':np.str,
             'width':np.float32,
             'height':np.float32,
             'class':np.str,
             'xmin':np.float32,
             'ymin':np.float32,
             'xmax':np.float32,
             'ymax':np.float32}

In [9]:
image_data = pd.read_csv(annotation_train_path, header=0, dtype=dtype_dict)

In [10]:
grouped = image_data.groupby('filename')

In [11]:


for name,group in grouped:
    image = cv2.imread(image_path + name)
    img_y,img_x,c = image.shape
    box_h = 400
    box_w = 260
    
    b_bb = []
    for index,bbox in group.iterrows():
        bb = BoundingBox(x1=bbox['xmin'], y1=bbox['ymin'], x2=bbox['xmax'], y2=bbox['ymax'], label= bbox['class'])
        b_bb.append(bb)
        
    rand_x = [i for i in range(0,img_x - box_h,int(img_x/40))]
    rand_y = [i for i in range(0,img_y - box_w,int(img_y/40))]
    index = 0
    for i in rand_x:
        for j in rand_y:
            t = BoundingBox(x1=i, y1=j, x2=i+box_w, y2=j+box_h, label= 'backgroud')
            iou0 = 0
            for j,b in enumerate(b_bb):
                t_iou = t.iou(b)
                if(t_iou>iou0):
                    iou0 = t_iou            
            if(iou0<0.1):
                index += 1
                bird = t.extract_from_image(image)
#                 cv2.imwrite(output_path + '0/' + name[:-4] + '_' + str(index) + '.jpg',bird)   #输出量大，谨慎执行 ！！！